In [1]:

import sys
import os
import glob
sys.path.append("../NEAT")
from NEATModels import NEATDynamic, nets
from NEATModels.config import dynamic_config
from NEATUtils import helpers
from NEATUtils.helpers import load_json
from stardist.models import StarDist2D
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
from pathlib import Path

Using TensorFlow backend.


In [2]:
imagedir = '/home/sancere/Kepler/FinalONEATTraining/NEATTest/'
model_dir = '/home/sancere/Kepler/FinalONEATTraining/EverydayneatmodelV2/'
savedir= '/home/sancere/Kepler/FinalONEATTraining/NEATTest/Save/'
model_name = 'divisionm4d65V2'
star_model_name = 'bin2stardist'
division_categories_json = model_dir + 'DivisionCategories.json'
catconfig = load_json(division_categories_json)
division_cord_json = model_dir + 'DivisionCord.json'
cordconfig = load_json(division_cord_json)
model = NEATDynamic(None, model_dir , model_name,catconfig, cordconfig)
starmodel = StarDist2D(config = None, name = star_model_name, basedir = model_dir)
Path(savedir).mkdir(exist_ok=True)
n_tiles = (1,1)
event_threshold = 0.9
iou_threshold = 0.6
yolo_v2 = True

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Loading network weights from 'weights_last.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.65, nms_thresh=0.075.


# In the code block below compute the markers and make a dictionary for each image

In [3]:
Raw_path = os.path.join(imagedir, '*tif')
X = glob.glob(Raw_path)
marker_dict = {}
for imagename in X:
     markers, markers_tree, density_location =  model.markers(imagename, starmodel, n_tiles = n_tiles)
     
     marker_dict[imagename] = [markers, markers_tree, density_location]
     

  0%|          | 0/20 [00:00<?, ?it/s]

Obtaining Markers



 50%|█████     | 10/20 [00:00<00:00, 95.26it/s]

Creating Dictionary of marker location for fast search


 25%|██▌       | 5/20 [00:00<00:00, 41.08it/s]

Computing density of each marker


100%|██████████| 20/20 [00:00<00:00, 37.99it/s]


# In the cell below use the computed dictionary and apply division model detection

In [4]:
Raw_path = os.path.join(imagedir, '*tif')
X = glob.glob(Raw_path)
for imagename in X:
     
     markers, markers_tree, density_location = marker_dict[imagename]
     model.predict(imagename,markers, markers_tree, density_location, savedir, n_tiles = n_tiles, event_threshold = event_threshold, iou_threshold = iou_threshold)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


  0%|          | 0/10 [00:00<?, ?it/s]

Detecting negative events to exclude from prediction


  0%|          | 0/20 [00:00<?, ?it/s]

Detecting event locations
{'Normal': 7.0495787e-09, 'Division': 1.0, 'xstart': 16, 'ystart': 68, 'xcenter': 40, 'ycenter': 105, 'real_time_event': 4.0, 'box_time_event': 0.0, 'height': 35.75686264038086, 'width': 29.539642333984375, 'confidence': 0.9846405982971191, 'realangle': 0.31810608257804435, 'rawangle': 0.6012563109397888}
{'Normal': 4.8292004e-10, 'Division': 1.0, 'xstart': 20, 'ystart': 68, 'xcenter': 46, 'ycenter': 84, 'real_time_event': 4.0, 'box_time_event': 0.0, 'height': 31.446985244750977, 'width': 31.765506744384766, 'confidence': 0.976947546005249, 'realangle': 1.0931723460789622, 'rawangle': 0.8479675650596619}
{'Normal': 1.892424e-09, 'Division': 1.0, 'xstart': 24, 'ystart': 68, 'xcenter': 58, 'ycenter': 107, 'real_time_event': 4.0, 'box_time_event': 0.0, 'height': 28.83919906616211, 'width': 27.824316024780273, 'confidence': 0.9981628656387329, 'realangle': 0.5414225637601426, 'rawangle': 0.6723401546478271}
{'Normal': 1.3929269e-08, 'Division': 1.0, 'xstart': 16, 

...

 50%|█████     | 10/20 [01:50<01:52, 11.28s/it]

{'Normal': 3.441516e-08, 'Division': 1.0, 'xstart': 9.0, 'ystart': 130.0, 'xcenter': 41.0, 'ycenter': 162.0, 'real_time_event': 13.0, 'box_time_event': 0.0, 'height': 20.1001033782959, 'width': 22.62795066833496, 'confidence': 0.8244107961654663, 'realangle': 0.42239767629582503, 'rawangle': 0.6344533562660217}
{'Normal': 4.246656e-09, 'Division': 1.0, 'xstart': 14.0, 'ystart': 185.0, 'xcenter': 46.0, 'ycenter': 217.0, 'real_time_event': 13.0, 'box_time_event': 0.0, 'height': 23.313907623291016, 'width': 34.52931594848633, 'confidence': 0.9994770884513855, 'realangle': -0.9180612392441972, 'rawangle': 0.20777203142642975}
{'Normal': 5.9984244e-14, 'Division': 1.0, 'xstart': 23.0, 'ystart': 142.0, 'xcenter': 55.0, 'ycenter': 174.0, 'real_time_event': 13.0, 'box_time_event': 0.0, 'height': 25.318458557128906, 'width': 27.963336944580078, 'confidence': 0.9999840259552002, 'realangle': -0.40929798220669955, 'rawangle': 0.3697164058685303}
{'Normal': 1.9289327e-08, 'Division': 1.0, 'xstart'

 55%|█████▌    | 11/20 [02:02<01:39, 11.11s/it]

{'Normal': 1.12853384e-13, 'Division': 1.0, 'xstart': 8.0, 'ystart': 130.0, 'xcenter': 40.0, 'ycenter': 162.0, 'real_time_event': 14.0, 'box_time_event': 0.0, 'height': 22.01637077331543, 'width': 25.300512313842773, 'confidence': 0.8885721564292908, 'realangle': -0.6241320664514893, 'rawangle': 0.30133259296417236}
{'Normal': 7.3595513e-10, 'Division': 1.0, 'xstart': 23.0, 'ystart': 142.0, 'xcenter': 55.0, 'ycenter': 174.0, 'real_time_event': 14.0, 'box_time_event': 0.0, 'height': 26.809558868408203, 'width': 24.07681655883789, 'confidence': 0.999414324760437, 'realangle': -0.4532602387368587, 'rawangle': 0.3557227849960327}
{'Normal': 1.6676463e-08, 'Division': 1.0, 'xstart': 63.0, 'ystart': 142.0, 'xcenter': 95.0, 'ycenter': 174.0, 'real_time_event': 14.0, 'box_time_event': 0.0, 'height': 23.887619018554688, 'width': 25.923341751098633, 'confidence': 0.9961658716201782, 'realangle': -0.3448657332706207, 'rawangle': 0.3902258276939392}
{'Normal': 3.1052828e-06, 'Division': 0.9999969,

KeyError: 'Division'

In [ ]:

model.showNapari(imagedir, savedir, yolo_v2)
